In [ ]:
import pandas as pd 
import numpy as np
from pandas_datareader import data as pdr
!pip install yfinance
import yfinance as yf #yahoo finance stock data
import datetime
import matplotlib.pyplot as plt
from math import sqrt

import tensorflow
from tensorflow import keras

from statsmodels.tsa.stattools import adfuller

from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.ensemble import RandomForestRegressor

stock="GOOG"
period=14

In [ ]:
end=datetime.datetime(2019,6,15,0,0,0)

In [ ]:
min=yf.download(tickers=stock,period="3d",enddate=end,interval="1m")
thirt_min=yf.download(tickers=stock,period="60d",enddate=end, interval="30m")
hour=yf.download(tickers=stock,period="150d",enddate=end,interval="1h")
daily=yf.download(tickers=stock,period="1000d",enddate=end,interval="1d")
weekly=yf.download(tickers=stock,period="3800d",enddate=end,interval="1wk")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
def Prepare_dataset(df):
    from sklearn.model_selection import train_test_split

    typ_price=(df.Close + df.High+df.Low)/3
    mf=typ_price*df.Volume

    pos_flow=[]
    neg_flow=[]

    for i in range(0,len(typ_price)):
        if typ_price[i]>typ_price[i-1]:
            pos_flow.append(mf[i-1])
            neg_flow.append(0)
        elif typ_price[i]<typ_price[i-1]:
            pos_flow.append(0)
            neg_flow.append(mf[i-1])
        else:
            pos_flow.append(0)
            neg_flow.append(0)
        
    pos_mf=[]
    neg_mf=[]

    for i in range(period-1,len(pos_flow)):
        pos_mf.append(sum(pos_flow[i+1-period:i]))

    for i in range(period-1,len(neg_flow)):
        neg_mf.append(sum(neg_flow[i+1-period:i]))

    mf_ratio=np.array(pos_mf)/np.array(neg_mf)

    mfi=100-100/(1+mf_ratio)
    df=df.iloc[13:,:]

    df["MFI"]=mfi

    df.dropna(inplace=True,how="all",axis=0)

    y=df.iloc[:-1,:]["Adj Close"].shift(-1)
    y.fillna(method='ffill', inplace=True)
    y=y.iloc[-755:]
    X=df.iloc[:-1,:].fillna(method='ffill')
    X=X.iloc[-755:]
    return train_test_split(X, y, test_size=0.4, random_state=0)

def Rmse(y,y_pred):
    return np.sqrt(((y_pred - y) ** 2).mean())
  
def Arima_build(ts_train,ts_test):

  def arimamodel(timeseriesarray):
    import pmdarima as pmd
    autoarima_model = pmd.auto_arima(timeseriesarray, 
                              start_p=1,
                              max_p=5,
                              start_q=1,
                              max_q=5,
                              test="adf",
                              max_order=5,
                              seasonal_test="OCSB",
                              stepwise=False,
                              n_jobs=-1,
                              trace=True)
    return autoarima_model

  arima_model = arimamodel(ts_train)
  preds=arima_model.predict(len(ts_test))
  return ts_test


In [ ]:
datasets={}
datasets["1m"]=Prepare_dataset(min)
datasets["30m"]=Prepare_dataset(thirt_min)
datasets["1h"]=Prepare_dataset(hour)
datasets["D"]=Prepare_dataset(daily)
datasets["W"]=Prepare_dataset(weekly)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
result=pd.DataFrame(columns=["1m","30m","1h","D","W"])

for key in datasets:
    X_train=datasets[key][0]
    X_test=datasets[key][1]
    y_train=datasets[key][2]
    y_test=datasets[key][3]

    elastic=ElasticNet().fit(X_train, y_train)
    elastic_preds=elastic.predict(X_test)

    result.loc["Elastic net",key]=[Rmse(y_test,elastic_preds)]

    rf=RandomForestRegressor(n_estimators=100,max_depth=12,random_state=0).fit(X_train, y_train)
    rf_preds=rf.predict(X_test)

    result.loc["Random forest",key]=[Rmse(y_test,rf_preds)]

    arima_preds=Arima_build(datasets[key][0]["Close"],datasets[key][1]["Close"])

    result.loc["Arima",key]=[Rmse(arima_preds,rf_preds)]

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 114.78583397705165, tolerance: 51.5743301150881
  positive)


Total fit time: 11.516 seconds


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 238.4765068784909, tolerance: 186.03852314214794
  positive)


Total fit time: 11.597 seconds


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9018.930257062086, tolerance: 222.76694316851308
  positive)


Total fit time: 11.637 seconds


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 118884.2718693269, tolerance: 1307.486914894378
  positive)


Total fit time: 11.588 seconds


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 150275.9145895186, tolerance: 7203.100387525364
  positive)


Total fit time: 11.692 seconds


In [ ]:
result

,1m,30m,1h,D,W
Elastic net,[6.38428395286416],[8.978426893072285],[11.483670620161837],[21.533698276421553],[22.564461634331725]
Random forest,[6.319696658781507],[9.014506226041245],[11.835937329122274],[24.4550716516014],[25.27843993152649]
Arima,[1.5295954726681689],[3.8187748356870785],[4.994953684383751],[10.405843595598585],[13.52952891509512]
